#<strong><center>Allripe</center></strong> 
#<strong><center>Pomologist ChatBot- Deep Learning</center></strong> 

## Data?
We will be using our own dataset for this Pomologist chatterbot.

In [ ]:
import os
import cv2
import numpy as np
from os.path import join
import matplotlib.pyplot as plt
from google.colab import drive


drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/content/drive/My Drive/Allripe/Chat Bot/convo_data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/drive/My Drive/Allripe/Chat Bot/convo_data/greetings.yml
/content/drive/My Drive/Allripe/Chat Bot/convo_data/botprofile.yml


In [ ]:
# Place this before directly or indirectly importing tensorflow
import logging
logging.getLogger("tensorflow").setLevel(logging.WARNING)

import tensorflow as tf
from tensorflow.keras import layers, preprocessing, utils
import yaml

print( tf.version)

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.6/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [ ]:
questions=list()
answers=list()

for dirname, _, filenames in os.walk('/content/drive/My Drive/Allripe/Chat Bot/convo_data/'):
    for filename in filenames:
        file = open(os.path.join(dirname, filename), 'rb')
        docs = yaml.safe_load(file)
        conversations = docs['conversations']
        for con in conversations:
            if len(con)>2:
                questions.append(con[0])
                replies=con[1:]
                ans=""
                for rep in replies:
                    ans+=' '+rep
                answers.append(ans)
            elif len(con)>1:
                questions.append(con[0])
                answers.append(con[1])
                
answers_with_tags=list()
for i in range( len(answers)):
    if type(answers[i])==str:
        answers_with_tags.append(answers[i])
    else:
        questions.pop(i)

answers=list()
for ans in answers_with_tags:
    answers.append('<START> '+ans+'. <END>')

tokenizer=preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions+answers)
VOCAB_SIZE= len(tokenizer.word_index)+1
print('VOCAB SIZE : {}'.format(VOCAB_SIZE))

VOCAB SIZE : 211


In [ ]:
from gensim.models import Word2Vec
import re

vocab=[]
for word in tokenizer.word_index:
    vocab.append(word)
    
def tokenize(sentences):
    tokens_list=[]
    vocabulary=[]
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub( '^a-zA-Z',' ',sentence)
        tokens = sentence.split()
        vocabulary+=tokens
        tokens_list.append(tokens)
    return tokens_list,vocabulary

p=tokenize(questions+answers)
model = Word2Vec(p[0])
'''
embedding_matrix = np.zeros((VOCAB_SIZE,100))
for i in range(len(tokenizer.word_index)):
    embedding_matrix[i] = model[vocab[i]]
   ''' 
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(x) for x in tokenized_questions])
padded_questions = preprocessing.sequence.pad_sequences(tokenized_questions, maxlen=maxlen_questions, padding='post')
encoder_input_data = np.array( padded_questions )
print(encoder_input_data.shape, maxlen_questions)

tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = max([len(x) for x in tokenized_answers])
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')
decoder_input_data = np.array(padded_answers)
print(decoder_input_data.shape, maxlen_answers)

tokenized_answers = tokenizer.texts_to_sequences(answers)
for i in range(len(tokenized_answers)):
    tokenized_answers[i]=tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')
one_hot_answers = utils.to_categorical(padded_answers, VOCAB_SIZE)
decoder_output_data = np.array(one_hot_answers)
print(decoder_output_data.shape)

(42, 16) 16
(42, 49) 49
(42, 49, 211)


In [ ]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 200)    42200       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 200)    42200       input_2[0][0]                    
_______________________________________________________________________________________

In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=10, epochs=500 ) 
model.save( 'model.h5' )

Epoch 1/500
5/5 [==============================] - 0s 9ms/step - loss: 1.0858 - accuracy: 0.1005
Epoch 2/500
5/5 [==============================] - 0s 9ms/step - loss: 1.0786 - accuracy: 0.1938
Epoch 3/500
5/5 [==============================] - 0s 8ms/step - loss: 1.0544 - accuracy: 0.1914
Epoch 4/500
5/5 [==============================] - 0s 8ms/step - loss: 1.0140 - accuracy: 0.1699
Epoch 5/500
5/5 [==============================] - 0s 9ms/step - loss: 0.9689 - accuracy: 0.1459
Epoch 6/500
5/5 [==============================] - 0s 8ms/step - loss: 0.9369 - accuracy: 0.1292
Epoch 7/500
5/5 [==============================] - 0s 8ms/step - loss: 0.9164 - accuracy: 0.1244
Epoch 8/500
5/5 [==============================] - 0s 9ms/step - loss: 0.8947 - accuracy: 0.1388
Epoch 9/500
5/5 [==============================] - 0s 8ms/step - loss: 0.8716 - accuracy: 0.1699
Epoch 10/500
5/5 [==============================] - 0s 8ms/step - loss: 0.8518 - accuracy: 0.1770
Epoch 11/500
5/5 [===========

In [ ]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [ ]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

In [ ]:
enc_model , dec_model = make_inference_models()
new_ques=[]
print("ROBO: My name is Robo. I will answer your queries about Allripe. If you want to exit, type Bye!")
for _ in range(1000000):
  question=input( 'Enter your question: ' )

  remove_characters = ["?", ".",",","!"]
  for character in remove_characters:
    question = question.replace(character, "")

  if question.lower()=="bye":
    print("Bye! Have a nice day!")
    print("The new unamswered questions are: {}".format(new_ques))
    break

  else:
    try:
      states_values = enc_model.predict( str_to_tokens( question ) )
    except:
      new_ques.append(question)
      print("I'm not programmed for this question, please enter another question.")
      continue
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
          stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )

ROBO: My name is Robo. I will answer your queries about Allripe. If you want to exit, type Bye!
Enter your question: hi
 hello end
Enter your question: hey buddy
 hi end
Enter your question: who are you?
 i am a pomologist i usually share food science information end
Enter your question: who you?
 i am a pomologist i mainly focus on sharing fruit science information end
Enter your question: how may i address you?
 i am a pomologist end
Enter your question: how i address you?
 i am a pomologist end
Enter your question: what is pomology
 pomological mainly focuses on the development enhancement cultivation and physiological studies of fruit trees the goals of fruit tree improvement include enhancement of fruit quality regulation of production periods and reduction of production cost one involved in the science of pomology is called a pomologist end
Enter your question: pomology?
 pomological mainly focuses on the development enhancement cultivation and physiological studies of fruit tree